# Otto商品分类——XGBoost，测试
原始特征

我们以Kaggle 2015年举办的Otto Group Product Classification Challenge竞赛数据为例。

Otto数据集是著名电商Otto提供的一个多类商品分类问题，类别数=9. 每个样本有93维数值型特征（整数，表示某种事件发生的次数，已经进行过脱敏处理）。 竞赛官网：https://www.kaggle.com/c/otto-group-product-classification-challenge/data


第一名：https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335
第二名：http://blog.kaggle.com/2015/06/09/otto-product-classification-winners-interview-2nd-place-alexander-guschin/

In [1]:
# 首先 import 必要的模块
import pandas as pd 
import numpy as np

## 读取数据 & 数据探索

In [2]:
# 读取数据
# 请自行在log(x+1)特征和tf_idf特征上尝试，并比较不同特征的结果，
# 我们可以采用stacking的方式组合这几种不同特征编码的得到的模型
# path to where the data lies
dpath = './data/'
test = pd.read_csv(dpath +"Otto_test.csv")
#test = pd.read_csv(dpath +"Otto_FE_test_log.csv")
#test2 = pd.read_csv(dpath +"Otto_FE_test_tfidf.csv")

#去掉多余的id
#test2 = test2.drop(["id"], axis=1)
#test =  pd.concat([test1, test2], axis = 1, ignore_index=False)
test.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,...,0,0,11,1,20,0,0,0,0,0
1,2,2,2,14,16,0,0,0,0,0,...,0,0,0,0,0,4,0,0,2,0
2,3,0,1,12,1,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,0,...,0,3,1,0,0,0,0,0,0,0
4,5,1,0,0,1,0,0,1,2,0,...,0,0,0,0,0,0,0,9,0,0


## 准备数据

In [3]:
test_id = test['id']   
X_test = test.drop(["id"], axis=1)

#保存特征名字以备后用（可视化）
feat_names = X_test.columns 

#sklearn的学习器大多之一稀疏数据输入，模型训练会快很多
#查看一个学习器是否支持稀疏数据，可以看fit函数是否支持: X: {array-like, sparse matrix}.
#可自行用timeit比较稠密数据和稀疏数据的训练时间
from scipy.sparse import csr_matrix
X_test = csr_matrix(X_test)

In [4]:
#load训练好的模型
import cPickle

model = cPickle.load(open("Otto_XGBoost_org.pkl", 'rb'))

#输出每类的概率
y_test_pred = model.predict_proba(X_test)

In [5]:
y_test_pred.shape

(144368, 9)

In [5]:
#生成提交结果
out_df = pd.DataFrame(y_test_pred)

columns = np.empty(9, dtype=object)
for i in range(9):
    columns[i] = 'Class_' + str(i+1)

out_df.columns = columns

out_df = pd.concat([test_id,out_df], axis = 1)
out_df.to_csv("XGBoost_org.csv", index=False)

原始特征和tfidf两种特征
Logistic回归：在Kaggle的Private Leaderboard分数0.59817（排名第2243位）
RBF核SVM（只有tfidf特征）：0.48947（排名1254位）
CART：1.07144（交叉验证估计的测试误差很准，可惜性能太差），单棵决策树性能不好
RF：0.52848
XGBoost:0.44729（只用原始特征，排名第636位）
LightGBM：0.44366（排名567位）